In [19]:
# Veri kümesine erişmek için Google Drive'a bağlanma
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [20]:
# Gerekli kütüphaneleri içe aktarma
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


In [21]:
# Veri kümesindeki klasörleri gösterme
import os
print(os.listdir("/gdrive/My Drive/Banana_Ripeness_Classification"))

['unripe', 'ripe', 'rotten', 'overripe']


In [22]:
# Veri kümesinin depolandığı dizini ve muz kategorilerini tanımlama
DATADIR = '/gdrive/My Drive/Banana_Ripeness_Classification/'
CATEGORIES = ["overripe", "ripe", "rotten", "unripe"]
IMG_SIZE = 100

In [23]:
# Görüntülerden bir veri kümesi oluşturma işlevi
def create_all_data():
    all_data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                all_data.append([new_array, class_num])
            except Exception as e:
                pass
    return all_data

In [24]:
# Veri kümesini özelliklere ve etiketlere dönüştürme
all_data = create_all_data()
np.random.shuffle(all_data)

# Verileri özelliklere (X) ve etiketlere (y) ayırma
X = []
y = []

for features, label in all_data:
    X.append(features)
    y.append(label)

# Görüntüleri normalleştirme ve numpy dizilerine dönüştürme
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X / 255.0
y = np.array(y)

In [35]:
# Senaryo 2: %70 Train, %30 Test ve %10 Validation
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)


In [36]:
# Etiketleri one-hot kodlanmış biçime dönüştürme
y_train = to_categorical(y_train, num_classes=len(CATEGORIES))
y_val = to_categorical(y_val, num_classes=len(CATEGORIES))
y_test = to_categorical(y_test, num_classes=len(CATEGORIES))


In [37]:
# ImageNet üzerinde önceden eğitilmiş VGG16 modelini üst katmanı olmadan yükleme
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

for layer in base_model.layers:
    layer.trainable = False


In [38]:
# Yeni bir sıralı model oluşturma ve önceden eğitilmiş modeli ve özel katmanları ekleme
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(CATEGORIES), activation='softmax')
])

In [39]:
# Modeli Adam optimizer ve kategorik çapraz entropi kaybı ile derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [40]:
# Modeli eğitim verileri üzerinde eğitme ve doğrulama verileri üzerinde doğrulama
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))


Epoch 1/10
266/266 [==============================] - 264s 991ms/step - loss: 0.4558 - accuracy: 0.8343 - val_loss: 0.2585 - val_accuracy: 0.9163
Epoch 2/10
266/266 [==============================] - 257s 968ms/step - loss: 0.2442 - accuracy: 0.9137 - val_loss: 0.2684 - val_accuracy: 0.9089
Epoch 3/10
266/266 [==============================] - 258s 970ms/step - loss: 0.1997 - accuracy: 0.9320 - val_loss: 0.1727 - val_accuracy: 0.9417
Epoch 4/10
266/266 [==============================] - 258s 970ms/step - loss: 0.1743 - accuracy: 0.9378 - val_loss: 0.1621 - val_accuracy: 0.9523
Epoch 5/10
266/266 [==============================] - 258s 968ms/step - loss: 0.1498 - accuracy: 0.9475 - val_loss: 0.1576 - val_accuracy: 0.9555
Epoch 6/10
266/266 [==============================] - 258s 969ms/step - loss: 0.1372 - accuracy: 0.9508 - val_loss: 0.1548 - val_accuracy: 0.9523
Epoch 7/10
266/266 [==============================] - 258s 970ms/step - loss: 0.1342 - accuracy: 0.9521 - val_loss: 0.1529 -

In [41]:
# Modeli test kümesi üzerinde değerlendirme
test_loss, test_acc = model.evaluate(x_test, y_test)
print("\nTest accuracy: ", test_acc)

127/127 [==============================] - 110s 863ms/step - loss: 0.1167 - accuracy: 0.9612

Test accuracy:  0.9611770510673523


In [42]:
# Test seti üzerindeki tahminleri gösterme
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

127/127 [==============================] - 111s 870ms/step


In [43]:
# Metrikleri hesaplama
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

In [44]:
# Karışıklık Matrisi ve Sınıflandırma Raporu
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes)


In [45]:
# Metrikleri görüntüleme
print("Accuracy:", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)


Accuracy: 0.9611770524233432
Precision:  0.9611928367726147
Recall:  0.9611770524233432
F1 Score:  0.9611679670128039

Confusion Matrix:
 [[ 783   28   10    5]
 [  19 1125   25    4]
 [  28   20 1317    9]
 [   0    1    8  662]]

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.95      0.95       826
           1       0.96      0.96      0.96      1173
           2       0.97      0.96      0.96      1374
           3       0.97      0.99      0.98       671

    accuracy                           0.96      4044
   macro avg       0.96      0.96      0.96      4044
weighted avg       0.96      0.96      0.96      4044

